## Lecture 7: Convnets in TensorFlow
### 0. Overview
1. Playing with convolutions
2. Convolution support in TF
3. More MNIST!!!
4. Autoencoder
5. Interactive coding

### 1. Understanding convolutions
- [Chris Olah: understanding convolution](http://colah.github.io/posts/2014-07-Understanding-Convolutions/)

### 2. Convolution support in TF
#### 2.1 `tf.nn.conv2d`

    tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu = None , data_format = None , name = None)

- Input: Batch size x Height x Width x Channels
- Filter: Height x Width x Input Channels x Output Channels ( e.g. `[5, 5, 3, 64]`)
- Strides: 4 element 1-D tensor , strides in each direction ( often `[1, 1, 1, 1]` or `[1, 2, 2, 1]`)
- Padding: `'SAME'` or `'VALID'`
- Data_format: default to NHWC

Because of this property of convolution, we can do convolutions without training anything. We can simply choose a kernel and see how that kernel affects our image.
- example: the kernel often used for blurring an image
![blur](figures/07_01.png)
- other popular kernels: in the `kernels.py` file on the [class GitHub repository](https://github.com/chiphuyen/tf-stanford-tutorials)
- how to use them: in `07_basic_filters.py`


#### 2.2 other built-in convolutional operations
- more details in [official documentation](https://www.tensorflow.org/api_docs/)
- `conv2d`: Arbitrary filters that can mix channels together.
- `depthwise_conv2d`: Filters that operate on each channel independently.
- `separable_conv2d`: A depthwise spatial filter followed by a pointwise filter.

In this case, we hard code our kernels. When training a convnet, we don’t know what the values
for our kernels and therefore have to figure them out by learning them.

### 3. Convnet on MNIST
- see `07_convnet_mnist.py`
- two convolutional layers, each followed by a relu and a maxpool layers, and one fully connected layer
![conv_MNIST](figures/07_02.png)

### 4. Variable scope
- similar to namespace: e.g. A variable name ‘weights’ in variable scope ‘conv1’ will become ‘conv1-weights’
- common practice: create a variable scope for each layer
- use `tf.get_variable()` instead of `tf.Variable()`

        tf.get_variable(< name >, <shape> , <initializer>)
        
- If a variable with that name already exists in that variable scope, we use that variable. If a variable with that name doesn’t already exists in that variable scope, TensorFlow creates a new variable.
- Nodes in the same variable scope will be grouped together, and therefore you don’t have to use name scope any more
- delare variable scope:

        with tf.variable_scope('conv1') as scope:

            w = tf.get_variable('weights', [5, 5, 1, 32])
            b = tf.get_variable('biases', [32], initializer=tf.random_normal_initializer())
            conv = tf.nn.conv2d(images, w, strides=[1, 1, 1, 1], padding='SAME')
            conv1 = tf.nn.relu(conv + b, name=scope.name)

        with tf.variable_scope('conv2') as scope:

            w = tf.get_variable('weights', [5, 5, 32, 64])
            b = tf.get_variable('biases', [64], initializer=tf.random_normal_initializer())
            conv = tf.nn.conv2d(conv1, w, strides=[1, 1, 1, 1], padding='SAME')
            conv2 = tf.nn.relu(conv + b, name=scope.name)
    
- more resources in [official doc](https://www.tensorflow.org/programmers_guide/variable_scope)

### 5. Autoencoder


### 6. Codes
- also see [official tutorial](https://www.tensorflow.org/get_started/mnist/beginners)
- article on [understanding CNN](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/)
- [visual information theory](http://colah.github.io/posts/2015-09-Visual-Information/)

In [1]:
""" Using convolutional net on MNIST dataset of handwritten digit
(http://yann.lecun.com/exdb/mnist/)
"""
from __future__ import print_function

import os
import time 

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

N_CLASSES = 10
dir = os.getcwd() + "/data/mnist"

In [2]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets(dir, one_hot=True)

Extracting /Users/yvonne/Documents/postCampus/MOOC/2017_Stanford_Tensorflow/project/data/mnist/train-images-idx3-ubyte.gz
Extracting /Users/yvonne/Documents/postCampus/MOOC/2017_Stanford_Tensorflow/project/data/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/yvonne/Documents/postCampus/MOOC/2017_Stanford_Tensorflow/project/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/yvonne/Documents/postCampus/MOOC/2017_Stanford_Tensorflow/project/data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
# Step 2: Define paramaters for the model
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 50

In [4]:
# Step 3: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# We'll be doing dropout for hidden layer so we'll need a placeholder
# for the dropout probability too
# Use None for shape so we can change the batch_size once we've built the graph
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder") # image, not explore 2d structure here (later maybe)
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder") # label(one hot)

dropout = tf.placeholder(tf.float32, name='dropout')

In [5]:
# Step 4 + 5: create weights + do inference
# the model is conv -> relu -> pool -> conv -> relu -> pool -> fully connected -> softmax

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    # use the dynamic dimension -1
    
    images = tf.reshape(X, shape=[-1, 28, 28, 1])

    # create kernel variable of dimension [5, 5, 1, 32]
    # use tf.truncated_normal_initializer()
    
    kernel = tf.get_variable('kernel', [5, 5, 1, 32], initializer=tf.truncated_normal_initializer())

    # create biases variable of dimension [32]
    # use tf.random_normal_initializer()
    
    biases = tf.get_variable('biases', [32], initializer=tf.random_normal_initializer()) 

    # apply tf.nn.conv2d. strides [1, 1, 1, 1], padding is 'SAME'
    
    conv = tf.nn.conv2d(images, kernel, strides=[1, 1, 1, 1], padding='SAME')

    # apply relu on the sum of convolution output and biases
    
    conv1 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 28 x 28 x 32

with tf.variable_scope('pool1') as scope:
    # apply max pool with ksize [1, 2, 2, 1], and strides [1, 2, 2, 1], padding 'SAME'
    
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # output is of dimension BATCH_SIZE x 14 x 14 x 32

with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    kernel = tf.get_variable('kernels', [5, 5, 32, 64], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [64],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 14 x 14 x 64

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='SAME')

    # output is of dimension BATCH_SIZE x 7 x 7 x 64

with tf.variable_scope('fc') as scope:
    # use weight of dimension 7 * 7 * 64 x 1024
    input_features = 7 * 7 * 64
    
    # create weights and biases

    w = tf.get_variable('weights', [input_features, 1024], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [1024], initializer=tf.random_normal_initializer())

    # reshape pool2 to 2 dimensional
    pool2 = tf.reshape(pool2, [-1, input_features])

    # apply relu on matmul of pool2 and w + b
    
    fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')

    # apply dropout
    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')

with tf.variable_scope('softmax_linear') as scope:
    # this you should know. get logits without softmax
    # you need to create weights and biases

    w = tf.get_variable('weights', [1024, N_CLASSES], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [N_CLASSES], initializer=tf.random_normal_initializer())
    logits = tf.matmul(fc, w) + b

In [8]:
# Step 6: define loss function
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
with tf.name_scope('loss'):
    # you should know how to do this too
    
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    loss = tf.reduce_mean(entropy, name='loss')

In [9]:
# Step 7: define training op
# using gradient descent with learning rate of LEARNING_RATE to minimize cost
# don't forgot to pass in global_step

optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss, global_step=global_step)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./my_graph/mnist', sess.graph)
    ##### You have to create folders to store checkpoints
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    initial_step = global_step.eval()

    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)

    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([optimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)
    
    print("Optimization Finished!") # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    # test the model
    n_batches = int(mnist.test.num_examples/BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], 
                                        feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   
    
    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))